In [25]:
import os

### CHANGE path HERE
base_path_win = "C:\\Projects\\ature\\" # WINDOWS
base_path_lin = "/home/akhanal1/Spring2018/ature" # LINUX

In [24]:
if os.name == 'nt':
    base_path = base_path_win
else:
    base_path = base_path_lin

sep = os.sep
data_file_path = base_path + sep + 'data' + sep + 'DRIVE' + 'test' + sep + 'images'
mask_path = base_path + sep + 'data' + sep + 'DRIVE' + sep + 'test' + sep + 'mask'
ground_truth_path = base_path + sep + 'data' + sep + 'DRIVE' + sep + sep + 'test' + sep + '1st_manual'
log_path = base_path + sep + 'logs'
  
os.chdir(base_path)

from commons.IMAGE import Image
from commons.ImgLATTICE import Lattice
import preprocess.utils.img_utils as imgutils
from commons.MAT import Mat
from PIL import Image as IMG
import numpy as np
from commons import constants as const
import cv2
from preprocess.algorithms import fast_mst as fmst
import itertools as itr
from itertools import count
import matplotlib.pyplot as plt
from random import shuffle

In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
def run_segmentation(img_obj, lattice_obj, params):
    
    ##### Unpack all params
    SKELETONIZE_THRESHOLD, IMG_LATTICE_COST_ASSIGNMENT_ALPHA, IMG_LATTICE_COST_GABOR_IMAGE_CONTRIBUTION, SEGMENTATION_THRESHOLD = params
    
    ##### Create skeleton based on threshold
    img_obj.create_skeleton(threshold=SKELETONIZE_THRESHOLD, kernels=imgutils.get_chosen_skeleton_filter())
    seed_node_list = imgutils.get_seed_node_list(img_obj.img_skeleton)     

    ##### Run segmnetation
    graph = fmst.run_segmentation(image_object=img_obj,
                          lattice_object=lattice_obj,
                          seed_list=seed_node_list,
                          segmentation_threshold=SEGMENTATION_THRESHOLD,
                          alpha=IMG_LATTICE_COST_ASSIGNMENT_ALPHA,
                          img_gabor_contribution=IMG_LATTICE_COST_GABOR_IMAGE_CONTRIBUTION,
                          img_original_contribution=1.5-IMG_LATTICE_COST_GABOR_IMAGE_CONTRIBUTION)

In [15]:
def get_precision_recall_accuracy(segmented, truth):

    TP = 0 #True Positive
    FP = 0 #False Positive
    FN = 0 #False Negative
    TN = 0 #True Negative
    for i in range(0, segmented.shape[0]):
        for j in range(0, segmented.shape[1]):
            if segmented[i, j] == 255 and truth[i, j] == 255:
                TP+=1
            if segmented[i, j] == 255 and truth[i, j] == 0:
                FP+=1
            if segmented[i, j] == 0 and truth[i, j] == 255:
                FN += 1
            if segmented[i, j] == 0 and truth[i, j] == 0:
                TN += 1
    
    return TP / (TP + FP), TP / (TP + FN), (TP + TN)/(TP + FP + FN + TN)

In [16]:
def generate_precision_recall_plot(log_file):
    os.chdir(log_path)
    log = np.loadtxt('01_test.tif_result.csv', skiprows=1, delimiter=',')
    plt.title('Precision vs Recall plot')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.scatter(log[:,2], log[:,3], label='Recall')
    plt.xlim((0,1))
    plt.ylim((0,1))
    plt.savefig('01_test.tif_result.csv' + '.png')

In [17]:
data_file_path

'/home/akhanal1/Spring2018/ature/data/DRIVE/test/images'

In [18]:
############# ENTRY POINT HERE ###############
############################################
print('starting..')
SK_THRESHOLD_PARAMS = np.arange(40, 61, 20)
ALPHA_PARAMS = np.arange(5, 7, 0.5)
GABOR_CONTRIBUTION_PARAMS = np.arange(0.9, 1.4, 0.2)
SEGMENTATION_THRESHOLD_PARAMS = np.arange(8, 13, 0.5)

PARAMS_ITR = itr.product(SK_THRESHOLD_PARAMS, ALPHA_PARAMS, GABOR_CONTRIBUTION_PARAMS, SEGMENTATION_THRESHOLD_PARAMS)

PARAMS_COMBINATION = list(PARAMS_ITR)

shuffle(PARAMS_COMBINATION)

#### Work on all images in a directory
os.chdir(data_file_path)
for test_image in os.listdir(os.getcwd()):
    
    c = count()
    next(c)
    
    print('### WORKING ON: ' + test_image)
    ### Load image as array
    original = IMG.open(test_image)
    original = np.array(original.getdata(), np.uint8).reshape(original.size[1], original.size[0], 3)
    
    #### Load the corresponding mask as array
    os.chdir(mask_path)
    mask_file = test_image.split('_')[0] + '_test_mask.gif'
    mask = IMG.open(mask_file)
    mask = np.array(mask.getdata(), np.uint8).reshape(mask.size[1], mask.size[0], 1)[:,:,0]
    print("Mask loaded: " + mask_file)
    
    ### Dialate mask
    dialate_kern = np.array([
        [0.0, 0.0, 0.5, 0.0, 0.0],
        [0.0, 0.2, 1.0, 0.2, 0.0],
        [0.5, 1.0, 1.0, 1.0, 0.5],
        [0.0, 0.2, 1.0, 0.2, 0.0],
        [0.0, 0.0, 0.5, 0.0, 0.0],
    ], np.uint8)
    
    mask = cv2.erode(mask, dialate_kern, iterations=4)
    after_mask = cv2.bitwise_and(original[:,:,1], original[:,:,1], mask=mask)
    
#     IMG.fromarray(after_mask).show()
#     break

    img_obj = Image(image_arr=after_mask)
    
    #### Load ground truth segmented result as array
    os.chdir(ground_truth_path)
    ground_truth_file = test_image.split('_')[0] + '_manual1.gif'
    truth = IMG.open(ground_truth_file)
    truth = np.array(truth.getdata(), np.uint8).reshape(truth.size[1], truth.size[0], 1)[:,:,0]
    print("Ground truth loaded: " + ground_truth_file)
    
    
    img_obj.apply_bilateral()
    img_obj.apply_gabor(kernel_bank=imgutils.get_chosen_gabor_bank())
    print('Filter applied.')
    
#     IMG.fromarray(img_obj.img_gabor).show()
#     break
    
    lattice_obj = Lattice(image_arr_2d=img_obj.img_gabor)
    lattice_obj.generate_lattice_graph()
    print('Lattice created.')
    
    os.chdir(log_path)
    log_file_name = test_image + "_result.csv"
    log_file = open(log_file_name,'w')
    
    ### Write header
    log_file.write(
    'ITERATION,FSCORE,PRECISION,RECALL,ACCURACY,'\
    'SKELETONIZE_THRESHOLD,'\
    'IMG_LATTICE_COST_ASSIGNMENT_ALPHA,'\
    'IMG_LATTICE_COST_GABOR_IMAGE_CONTRIBUTION,'\
    'SEGMENTATION_THRESHOLD\n'
    )
    
    for params in PARAMS_COMBINATION:
        
        i = next(c)
        run_segmentation(img_obj, lattice_obj, params)
        
        precision, recall, accuracy = get_precision_recall_accuracy(lattice_obj.accumulator, truth)
        
        f1_score = 2 * precision * recall / (precision + recall) 
     
        line = str(i) + ',' + \
                str(round(f1_score, 3)) + ',' + \
                str(round(precision, 3)) + ',' + \
                str(round(recall, 3)) + ',' +\
                str(round(accuracy, 3)) + ',' +\
                ','.join(map(str, params))
                        
        log_file.write(line + '\n')
        
        log_file.flush()
        IMG.fromarray(lattice_obj.accumulator).save(test_image + '__[' + line + ']' + '.JPEG')
        print('Number of parameter combinations tried: ' + str(i), end='\r')
    
    log_file.close()
    os.chdir(log_path)
    generate_precision_recall_plot(log_file_name)

starting..
### WORKING ON: 17_test.tif
Mask loaded: 17_test_mask.gif
Ground truth loaded: 17_manual1.gif
Filter applied.
Lattice created.


KeyboardInterrupt: 

In [7]:
os.chdir(log_path)
log = np.loadtxt('01_test.tif_result.csv', skiprows=1, delimiter=',')
plt.title('Precision vs Recall plot')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.scatter(log[:,2], log[:,3], label='Recall')
plt.xlim((0,1))
plt.ylim((0,1))
plt.savefig('01_test.tif_result.csv' + '.png')


FileNotFoundError: [Errno 2] No such file or directory: '/home/akhanal1/Spring2018/ature/logs'

In [21]:
| = a

SyntaxError: invalid syntax (<ipython-input-21-0975c8db82c9>, line 1)